In [ ]:
#for installing the packages for the 1st time use !pip install [package name]
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#pip install spacy
import spacy
import re
from html import unescape

!pip install emoji
from emoji import UNICODE_EMOJI

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix


import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import datetime

import pickle

!pip install spacy

!pip install scipy
import scipy.sparse as sparse

from joblib import Parallel, delayed

In [ ]:
dftrain = pd.read_csv("C:/Users/flore/Desktop/UTS S5/STEM fellowship/sentiment140data/training.data.csv", encoding = "latin-1",names=["predictions","id","timestamp", "query", "user","text"])

In [ ]:
dftrain

In [ ]:
dftest = pd.read_csv("C:/Users/flore/Desktop/UTS S5/STEM fellowship/sentiment140data/testdata.manual.2009.06.14.csv", encoding = "latin-1",names=["predictions","id","timestamp", "query", "user","text"])

In [ ]:
dftest

In [ ]:
df_chopsticks = pd.read_csv("C:/Users/flore/Desktop/UTS S5/STEM fellowship/dolcegabbanadata/dolcegabbana_chopsticks_mentions_daily_expanded.csv")

In [ ]:
df_all = pd.read_csv("C:/Users/flore/Desktop/UTS S5/STEM fellowship/dolcegabbanadata/dolcegabbana_mentions_daily_all.csv",   lineterminator='\n')

In [ ]:
df_chopsticks

In [ ]:
df_all

In [ ]:
# load NLP model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# helper function for pre-processing/cleaning a tweet
def preprocessor(tweet):
    tweet = re.sub (r'@[A-Za-z0-9_]+', '_AT_USER_', tweet) # replace @X with _AT_USER_
    tweet = re.sub (r'#[A-Za-z0-9_]+', '_HASHTAG_', tweet) # replace #X with _HASTHAG_
    tweet = re.sub (r'^RT[\s]+', '', tweet) # remove RT (retweet) at the start of the tweet
    tweet = unescape(tweet) # unescape the HTML
    tweet = tweet.lower() # make everything lowercase
    return tweet

# helper function for tokenization of a tweet
def tokenizer(tweet):
    tokens = nlp(tweet) # this processes the tweet text

    # tokenize each tweet and remove stop words and punctuation 
    tokens = [token for token in tokens if not token.is_stop and not token.is_punct] 
    
    # only keep tokens that are alphanumeric (including "-" and "_") or represent an emoji
    tokens = [t for t in tokens if re.match("^[a-zA-Z0-9_-]*$", t.text) or t.text in UNICODE_EMOJI]

    # lematize tokens
    tokens = [token.lemma_ for token in tokens]

    return tokens

In [ ]:
corpus_chopsticks = list(df_chopsticks['text']) # a list of tweets
corpus_chopsticks
corpus_all = list(df_all['text']) # a list of tweets
corpus_all
corpus_test = list(dftest['text']) # a list of tweets
corpus_test
corpus_train = list(dftrain['text']) # a list of tweets
corpus_train

In [ ]:
y_train = list(dftrain['predictions'])
y_test = list(dftest['predictions'])

## SVM
manually: (1)

In [ ]:
model = CountVectorizer(preprocessor=preprocessor, tokenizer=tokenizer, max_features=5000)
word_counts_train = model.fit_transform(corpus_train)
tokens_train = model.get_feature_names()
fitted_model=model

In [ ]:
model_features = model.get_feature_names()

In [ ]:
word_counts_test = fitted_model.transform(corpus_test)

In [ ]:
word_counts_chopsticks = fitted_model.transform(corpus_chopsticks)
word_counts_all = fitted_model.transform(corpus_all)

In [ ]:
fitted_model

In [ ]:
# use TF-IDF transformation
tfidf_transformer = TfidfTransformer()
X_140 = tfidf_transformer.fit_transform(word_counts_train) # training
X_test = tfidf_transformer.transform(word_counts_test) # testing
X_chopsticks = tfidf_transformer.transform(word_counts_chopsticks) # training
X_all =  tfidf_transformer.transform(word_counts_all) 

In [ ]:
X_140

In [ ]:
X_train, X_validation, y_trainsmall, y_validation = train_test_split(X_140, y_train, test_size=0.10, random_state=42, shuffle=True)

In [ ]:
print(X_train.shape)
print(X_validation.shape)
print(X_test.shape)

loading: (1)

In [ ]:
fitted_model = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/model_path", 'rb'))

model_features = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/model_features.npy', allow_pickle=True)

word_counts_test = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/word_counts_test.npz')
word_counts_chopsticks = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/word_counts_chopsticks.npz')
word_counts_all = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/word_counts_all.npz')
word_counts_train = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/word_counts_train.npz')

X_140 = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/X_140.npz')
X_test = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/X_test.npz')
X_chopsticks = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/X_chopsticks.npz')
X_all = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/X_all.npz')

X_train = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/X_train.npz')
X_validation = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/X_validation.npz')
y_trainsmall = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_trainsmall.npy', allow_pickle=True)
y_validation = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_validation.npy', allow_pickle=True)
y_test = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_test.npy', allow_pickle=True)

In [ ]:
X_validation

manual svc: (2)

In [ ]:
# fit SVC model with default parameters
svc_model = SVC(kernel='linear', C=0.1, probability=True)
svc_model.fit(X_train, y_trainsmall)

In [ ]:
# prediction on training
y_train_pred_svc = svc_model.predict(X_train)
y_train_proba_svc = svc_model.predict_proba(X_train)

# prediction on validation
y_validation_pred_svc = svc_model.predict(X_validation)
y_validation_proba_svc = svc_model.predict_proba(X_validation)

# prediction on chopsticks
y_chopsticks_pred_svc = svc_model.predict(X_chopsticks)
y_chopsticks_proba_svc = svc_model.predict_proba(X_chopsticks)

# prediction on d&g overall
y_all_pred_svc = svc_model.predict(X_all)
y_all_proba_svc = svc_model.predict_proba(X_all)

In [ ]:
print('Training performance')
print(classification_report(y_trainsmall, y_train_pred_svc))
print()
print('====================')
print()
print('Validation performance')
print(classification_report(y_validation, y_validation_pred_svc))

In [ ]:
plt.figure()
plt.hist(y_validation_proba_svc[:,1])
plt.xlabel('Predicted probability')
plt.ylabel('Number of tweets')
plt.title('Predicted probability distirbution training')
plt.show()

loading svc: (2)

In [ ]:
svc_model = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/svc_model", 'rb'))

y_train_pred_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_train_pred_svc.npy', allow_pickle=True)
y_train_proba_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_train_proba_svc.npy', allow_pickle=True)
y_validation_pred_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_validation_pred_svc.npy', allow_pickle=True)
y_validation_proba_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_validation_proba_svc.npy', allow_pickle=True)
y_chopsticks_pred_svc_binary = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_chopsticks_pred_svc_binary.npy', allow_pickle=True)
y_chopsticks_proba_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_chopsticks_proba_svc.npy', allow_pickle=True)
y_all_pred_svc_binary = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_all_pred_svc_binary.npy', allow_pickle=True)
y_all_proba_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_all_proba_svc.npy', allow_pickle=True)

manual 140test / probability: (3)

In [ ]:
# prediction on testing (once hyperparameters are tuned)
y_test_pred_svc_binary = svc_model.predict(X_test)
y_test_proba_svc = svc_model.predict_proba(X_test)

In [ ]:
plt.figure()
plt.hist(y_chopsticks_proba_svc[:,1])
plt.xlabel('Predicted probability')
plt.ylabel('Number of tweets')
plt.title('Predicted probability distribution validation')
plt.show()

In [ ]:
plt.figure()
plt.hist(y_all_proba_svc[:,1])
plt.xlabel('Predicted probability')
plt.ylabel('Number of tweets')
plt.title('Predicted probability distribution validation')
plt.show()

In [ ]:
y_chopsticks_proba_svc

In [ ]:
topacc = 0
top_a = 0
top_b = 0
for i in range (10):
    a = 0.5+(i*0.01)
    for i in range (10):
        y_test_pred_svc_binary = []
        b = 0.76-(i*0.01)
        for i in range (len(y_test_proba_svc)):
            if (y_test_proba_svc[i,1])<a:
                y_test_pred_svc_binary.append(0)
            elif(y_test_proba_svc[i,1])>b:
                y_test_pred_svc_binary.append(4)
            else:
                y_test_pred_svc_binary.append(2)
        accuracy = accuracy_score(y_test, y_test_pred_svc_binary)
        if accuracy > topacc:
            topacc = accuracy
            top_a = a
            top_b = b
            top_y_pred = y_test_pred_svc_binary
            print("acc = " + str(accuracy)+ " a = "+str(a)+" b = "+str(b))
            
        
print("overall top acc = "+str(topacc)+ " a = "+str(top_a)+" b = "+str(top_b))


In [ ]:
#best cutoff values: 0.5 + 0.69 (positive proba(second column))
y_test_pred_svc = top_y_pred

In [ ]:
print('Testing performance')
print(classification_report(y_test, y_test_pred_svc))

In [ ]:
y_chopsticks_pred_svc = []
for i in range (len(y_chopsticks_proba_svc)):
            if (y_chopsticks_proba_svc[i,1])<0.5:
                y_chopsticks_pred_svc.append(0)
            elif(y_chopsticks_proba_svc[i,1])>0.69:
                y_chopsticks_pred_svc.append(4)
            else:
                y_chopsticks_pred_svc.append(2)
y_all_pred_svc = []
for i in range (len(y_all_proba_svc)):
            if (y_all_proba_svc[i,1])<0.5:
                y_all_pred_svc.append(0)
            elif(y_all_proba_svc[i,1])>0.69:
                y_all_pred_svc.append(4)
            else:
                y_all_pred_svc.append(2)

In [ ]:
# see where model predicted wrong
for i in range(len(y_test_pred_svc)):
  if y_test[i] != y_test_pred_svc[i]:
    tweet = corpus_test[i]
    print(f'y_true {y_test[i]}, y_pred {y_test_pred_svc[i]}')
    print(tweet)
    print()

loading probability: (3)

In [ ]:
y_test_proba_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_test_proba_svc.npy', allow_pickle=True)
y_test_pred_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_test_pred_svc.npy', allow_pickle=True)
y_chopsticks_pred_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_chopsticks_pred_svc.npy', allow_pickle=True)
y_all_pred_svc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/y_all_pred_svc.npy', allow_pickle=True)

manually adding predictions to df: (4)

In [ ]:
df_all

In [ ]:
df_chopsticks

In [ ]:
y_chopsticks_proba_svc

In [ ]:
df_chopsticks['pred_label'] = y_chopsticks_pred_svc
df_all['pred_label'] = y_all_pred_svc

df_chopsticks['pred_score'] = y_chopsticks_proba_svc [:,1]
df_all[ 'pred_score'] = y_all_proba_svc [:,1]

In [ ]:
# Changing object type column to datetime
df_chopsticks['interval_start_date'] = pd.to_datetime(df_chopsticks['interval_start_date'])
df_chopsticks['created_at'] = pd.to_datetime(df_chopsticks['created_at'])
df_chopsticks['date'] = df_chopsticks['created_at'].dt.date # Creating new column with just the date
df_chopsticks['date'] = pd.to_datetime(df_chopsticks['date'])

# get week start date to aggregate over weeks
df_chopsticks['week_start_date'] = df_chopsticks['date'].apply(lambda x: x - datetime.timedelta(days=x.weekday()))
df_chopsticks['week_start_date'] = df_chopsticks['week_start_date'].dt.date
df_chopsticks.head()

# Changing object type column to datetime
df_all['interval_start_date'] = pd.to_datetime(df_all['interval_start_date'])
df_all['created_at'] = pd.to_datetime(df_all['created_at'])
df_all['date'] = df_all['created_at'].dt.date # Creating new column with just the date
df_all['date'] = pd.to_datetime(df_all['date'])

# get week start date to aggregate over weeks
df_all['week_start_date'] = df_all['date'].apply(lambda x: x - datetime.timedelta(days=x.weekday()))
df_all['week_start_date'] = df_all['week_start_date'].dt.date
df_all.head()

In [ ]:
# positive is going to be defined as greater than 80% prediction score that it is positive (adjustable based on threshold)
df_all_pos = df_all[(df_all['pred_label'] == 4)&(df_all['pred_score']>0.8)]

# negative is going to be defined as greater than 80% prediction score that it is negative (adjustable based on threshold)
df_all_neg = df_all[(df_all['pred_label'] == 0)&(df_all['pred_score']<0.4)]

# positive is going to be defined as greater than 80% prediction score that it is positive (adjustable based on threshold)
df_chopsticks_pos = df_chopsticks[(df_chopsticks['pred_label'] == 4)&(df_chopsticks['pred_score']>0.8)]

# negative is going to be defined as greater than 80% prediction score that it is negative (adjustable based on threshold)
df_chopsticks_neg = df_chopsticks[(df_chopsticks['pred_label'] == 0)&(df_chopsticks['pred_score']<0.2)]

In [ ]:
df_all_neg

loading predictions to df: (4)

In [ ]:
df_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/df_chopsticks_pred.pkl')
df_chopsticks_pos = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/df_chopsticks_pos.pkl')
df_chopsticks_neg = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/df_chopsticks_neg.pkl')

df_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/df_all_pred.pkl')
df_all_pos = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/df_all_pos.pkl')
df_all_neg = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/df_all_neg.pkl')

manually sorting by week: (5)

In [ ]:
# aggregate data and get aggregated statistics
pos_byweek_all = df_all_pos.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for positive tweets, get total retweets each week (sum of num retweets column over each week)
pos_byweek_all.columns = ['date', 'total_retweets']

neg_byweek_all = df_all_neg.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for negative tweets, get total retweets each week (sum of num retweets column over each week)
neg_byweek_all.columns = ['date', 'total_retweets']

# aggregate data and get aggregated statistics
pos_byweek_chopsticks = df_chopsticks_pos.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for positive tweets, get total retweets each week (sum of num retweets column over each week)
pos_byweek_chopsticks.columns = ['date', 'total_retweets']

neg_byweek_chopsticks = df_chopsticks_neg.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for negative tweets, get total retweets each week (sum of num retweets column over each week)
neg_byweek_chopsticks.columns = ['date', 'total_retweets']

In [ ]:
# for each week, get list of tweets with the highest number of retweets (which you can manually review to see what was trending that week)
top_tweets_peak_weeks_all = df_all.sort_values(by='retweet_count', ascending=False)
top_tweets_peak_weeks_all = top_tweets_peak_weeks.groupby('week_start_date').head(10).sort_values(by=['week_start_date', 'retweet_count'], ascending=False)
top_tweets_peak_weeks_all
#top_tweets_peak_weeks.to_csv(os.path.join(data_dir, 'top_tweets_peak_weeks.csv'), index=False)

In [ ]:
# for each week, get list of tweets with the highest number of retweets (which you can manually review to see what was trending that week)
top_tweets_peak_weeks_chopsticks = df_chopsticks.sort_values(by='retweet_count', ascending=False)
top_tweets_peak_weeks_chopsticks = top_tweets_peak_weeks_chopsticks.groupby('week_start_date').head(10).sort_values(by=['week_start_date', 'retweet_count'], ascending=False)
top_tweets_peak_weeks_chopsticks
#top_tweets_peak_weeks.to_csv(os.path.join(data_dir, 'top_tweets_peak_weeks.csv'), index=False)

loading by week: (5)

In [ ]:
pos_byweek_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/pos_byweek_chopsticks.pkl')
neg_byweek_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/neg_byweek_chopsticks.pkl')
top_tweets_peak_weeks_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/top_tweets_peak_weeks_chopsticks.pkl')


pos_byweek_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/pos_byweek_all.pkl')
neg_byweek_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/neg_byweek_all.pkl')
top_tweets_peak_weeks_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/top_tweets_peak_weeks_all.pkl')


manual tweet #: (5b)

In [ ]:
countlist = []
for i in range (len(df_all_pos)):
    countlist.append(1)
    
df_all_pos['counting'] = countlist
df_all_pos

pos_byweek_tweets_all = df_all_pos.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
pos_byweek_tweets_all.columns = ['date', '# of tweets']
pos_byweek_tweets_all


In [ ]:
countlist = []
for i in range (len(df_chopsticks_pos)):
    countlist.append(1)
    
df_chopsticks_pos['counting'] = countlist
df_chopsticks_pos

pos_byweek_tweets_chopsticks = df_chopsticks_pos.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
pos_byweek_tweets_chopsticks.columns = ['date', '# of tweets']
pos_byweek_tweets_chopsticks

In [ ]:
countlist = []
for i in range (len(df_chopsticks_neg)):
    countlist.append(1)
    
df_chopsticks_neg['counting'] = countlist
df_chopsticks_neg

neg_byweek_tweets_chopsticks = df_chopsticks_neg.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
neg_byweek_tweets_chopsticks.columns = ['date', '# of tweets']
neg_byweek_tweets_chopsticks

In [ ]:
countlist = []
for i in range (len(df_all_neg)):
    countlist.append(1)
    
df_all_neg['counting'] = countlist
df_all_neg

neg_byweek_tweets_all = df_all_neg.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
neg_byweek_tweets_all.columns = ['date', '# of tweets']
neg_byweek_tweets_all

In [ ]:
pickle.dump(neg_byweek_tweets_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/neg_byweek_tweets_chopsticks.pkl", "wb")) 
pickle.dump(neg_byweek_tweets_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/neg_byweek_tweets_all.pkl", "wb")) 
pickle.dump(pos_byweek_tweets_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/pos_byweek_tweets_chopsticks.pkl", "wb")) 
pickle.dump(pos_byweek_tweets_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/pos_byweek_tweets_all.pkl", "wb")) 

loading tweet #: (5b)

In [ ]:
neg_byweek_tweets_chopsticks = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/neg_byweek_tweets_chopsticks.pkl", 'rb'))
neg_byweek_tweets_all = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/neg_byweek_tweets_all.pkl", 'rb'))
pos_byweek_tweets_chopsticks = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/pos_byweek_tweets_chopsticks.pkl", 'rb'))
pos_byweek_tweets_all = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/100k train/pos_byweek_tweets_all.pkl", 'rb'))

manually making time series graphs: (6)

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
ax.plot(pos_byweek_all["date"], pos_byweek_all['total_retweets'], c='blue', label='Positive tweets')
ax.plot(neg_byweek_all["date"], neg_byweek_all['total_retweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 10000]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
ax.plot(neg_byweek_all["date"], neg_byweek_all['total_retweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 1000]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()

samething but tweet #: (6b)

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
ax.plot(pos_byweek_tweets_all["date"], pos_byweek_tweets_all['# of tweets'], c='blue', label='Positive tweets')
ax.plot(neg_byweek_tweets_all["date"], neg_byweek_tweets_all['# of tweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 2000]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
#ax.plot(pos_byweek_tweets_chopsticks["date"], pos_byweek_tweets_chopsticks['# of tweets'], c='blue', label='Positive tweets')
ax.plot(neg_byweek_tweets_all["date"], neg_byweek_tweets_all['# of tweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 250]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()

## RFC
manually tokenizing:

In [ ]:
model = CountVectorizer(preprocessor=preprocessor, tokenizer=tokenizer, max_features=5000)
word_counts_train = model.fit_transform(corpus_train)
tokens_train = model.get_feature_names()
fitted_model=model

In [ ]:
pickle.dump(fitted_model, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/fitted_model.pkl", "wb")) 
pickle.dump(model_features, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/model_features", 'wb'))

sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_train.npz', word_counts_train, compressed=True)
sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_chopsticks.npz', word_counts_chopsticks, compressed=True)
sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_test.npz', word_counts_test, compressed=True)
sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_all.npz', word_counts_all, compressed=True)

sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_140.npz', X_140, compressed=True)
sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_test.npz', X_test, compressed=True)
sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_chopsticks.npz', X_chopsticks, compressed=True)
sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_all.npz', X_all, compressed=True)

sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_train.npz', X_train, compressed=True)
sparse.save_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_validation.npz', X_validation, compressed=True)
pickle.dump(y_trainsmall, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_trainsmall", 'wb'))
pickle.dump(y_validation, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_validation", 'wb'))
pickle.dump(y_test, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_test", 'wb'))


In [ ]:
model_features = model.get_feature_names()

In [ ]:
word_counts_test = fitted_model.transform(corpus_test)

In [ ]:
word_counts_chopsticks = fitted_model.transform(corpus_chopsticks)
word_counts_all = fitted_model.transform(corpus_all)

In [ ]:
fitted_model

In [ ]:
# use TF-IDF transformation
tfidf_transformer = TfidfTransformer()
X_140 = tfidf_transformer.fit_transform(word_counts_train) # training
X_test = tfidf_transformer.transform(word_counts_test) # testing
X_chopsticks = tfidf_transformer.transform(word_counts_chopsticks) # training
X_all =  tfidf_transformer.transform(word_counts_all) 

In [ ]:
X_140

In [ ]:
X_train, X_validation, y_trainsmall, y_validation = train_test_split(X_140, y_train, test_size=0.10, random_state=42, shuffle=True)

In [ ]:
print(X_train.shape)
print(X_validation.shape)
print(X_test.shape)

loading tokenized data: (1)

In [ ]:
#loading vectorizer path
fitted_model = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/fitted_model.pkl", 'rb'))
model_features = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/model_features', allow_pickle=True)

word_counts_test = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_test.npz')
word_counts_chopsticks = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_chopsticks.npz')
word_counts_all = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_all.npz')
word_counts_train = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/word_counts_train.npz')

X_140 = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_140.npz')
X_test = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_test.npz')
X_chopsticks = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_chopsticks.npz')
X_all = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_all.npz')

X_train = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_train.npz')
X_validation = sparse.load_npz('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/X_validation.npz')
y_trainsmall = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_trainsmall', allow_pickle=True)
y_validation = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_validation', allow_pickle=True)
y_test = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_test', allow_pickle=True)


manual rfc: (2)

In [ ]:
rfc_model = RandomForestClassifier(min_samples_split=90)
#rfc_model = RandomForestClassifier(max_depth=10, min_samples_split=90, max_leaf_nodes=8, min_samples_leaf=7, n_estimators=110, max_samples=0.65, max_features=80, random_state=0)
rfc_model.fit(X_train, y_trainsmall)

In [ ]:
# prediction on training
y_train_pred_rfc = rfc_model.predict(X_train)
y_train_proba_rfc = rfc_model.predict_proba(X_train)

# prediction on validation
y_validation_pred_rfc = rfc_model.predict(X_validation)
y_validation_proba_rfc = rfc_model.predict_proba(X_validation)

# prediction on chopsticks
y_chopsticks_pred_rfc = rfc_model.predict(X_chopsticks)
y_chopsticks_proba_rfc = rfc_model.predict_proba(X_chopsticks)

# prediction on d&g overall
y_all_pred_rfc = rfc_model.predict(X_all)
y_all_proba_rfc = rfc_model.predict_proba(X_all)

In [ ]:
print('Training performance')
print(classification_report(y_trainsmall, y_train_pred_svc))
print()
print('====================')
print()
print('Validation performance')
print(classification_report(y_validation, y_validation_pred_svc))

In [ ]:
plt.figure()
plt.hist(y_validation_proba_rfc[:,1])
plt.xlabel('Predicted probability')
plt.ylabel('Number of tweets')
plt.title('Predicted probability distirbution training')
plt.show()

In [ ]:
pickle.dump(rfc_model, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/rfc_model.pkl", "wb")) 

pickle.dump(y_train_pred_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_train_pred_rfc", 'wb'))
pickle.dump(y_train_proba_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_train_proba_rfc", 'wb'))
pickle.dump(y_validation_pred_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_validation_pred_rfc", 'wb'))
pickle.dump(y_validation_proba_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_validation_proba_rfc", 'wb'))
pickle.dump(y_chopsticks_pred_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_chopsticks_pred_rfc_binary", 'wb'))
pickle.dump(y_chopsticks_proba_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_chopsticks_proba_rfc", 'wb'))
pickle.dump(y_all_pred_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_all_pred_rfc_binary", 'wb'))
pickle.dump(y_all_proba_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_all_proba_rfc", 'wb'))

loading rfc: (2)

In [ ]:
rfc_model = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/rfc_model.pkl", 'rb'))

y_train_pred_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_train_pred_rfc', allow_pickle=True)
y_train_proba_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_train_proba_rfc', allow_pickle=True)
y_validation_pred_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_validation_pred_rfc', allow_pickle=True)
y_validation_proba_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_validation_proba_rfc', allow_pickle=True)
y_chopsticks_pred_rfc_binary = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_chopsticks_pred_rfc_binary', allow_pickle=True)
y_chopsticks_proba_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_chopsticks_proba_rfc', allow_pickle=True)
y_all_pred_rfc_binary = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_all_pred_rfc_binary', allow_pickle=True)
y_all_proba_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_all_proba_rfc', allow_pickle=True)

manual 140test/probability (3):

In [ ]:
# prediction on testing (once hyperparameters are tuned)
y_test_pred_rfc_binary = rfc_model.predict(X_test)
y_test_proba_rfc = rfc_model.predict_proba(X_test)

In [ ]:
plt.figure()
plt.hist(y_chopsticks_proba_rfc[:,1])
plt.xlabel('Predicted probability')
plt.ylabel('Number of tweets')
plt.title('Predicted probability distribution validation')
plt.show()

In [ ]:
plt.figure()
plt.hist(y_all_proba_svc[:,1])
plt.xlabel('Predicted probability')
plt.ylabel('Number of tweets')
plt.title('Predicted probability distribution validation')
plt.show()

In [ ]:
y_chopsticks_proba_rfc

In [ ]:
topacc = 0
top_a = 0
top_b = 0
for i in range (10):
    a = 0.48+(i*0.01)
    for i in range (10):
        y_test_pred_rfc_binary = []
        b = 0.83-(i*0.01)
        for i in range (len(y_test_proba_rfc)):
            if (y_test_proba_rfc[i,1])<a:
                y_test_pred_rfc_binary.append(0)
            elif(y_test_proba_rfc[i,1])>b:
                y_test_pred_rfc_binary.append(4)
            else:
                y_test_pred_rfc_binary.append(2)
        accuracy = accuracy_score(y_test, y_test_pred_rfc_binary)
        if accuracy > topacc:
            topacc = accuracy
            top_a = a
            top_b = b
            top_y_pred = y_test_pred_rfc_binary
        print("acc = " + str(accuracy)+ " a = "+str(a)+" b = "+str(b))
            
        
print("overall top acc = "+str(topacc)+ " a = "+str(top_a)+" b = "+str(top_b))


In [ ]:
#best cutoff values: a = 0.48 b = 0.83 (positive proba(second column))
y_test_pred_rfc = top_y_pred

In [ ]:
print('Testing performance')
print(classification_report(y_test, y_test_pred_rfc))

In [ ]:
y_chopsticks_pred_rfc = []
for i in range (len(y_chopsticks_proba_rfc)):
            if (y_chopsticks_proba_rfc[i,1])<0.5:
                y_chopsticks_pred_rfc.append(0)
            elif(y_chopsticks_proba_rfc[i,1])>0.69:
                y_chopsticks_pred_rfc.append(4)
            else:
                y_chopsticks_pred_rfc.append(2)
y_all_pred_rfc = []
for i in range (len(y_all_proba_rfc)):
            if (y_all_proba_rfc[i,1])<0.5:
                y_all_pred_rfc.append(0)
            elif(y_all_proba_rfc[i,1])>0.69:
                y_all_pred_rfc.append(4)
            else:
                y_all_pred_rfc.append(2)

In [ ]:
# see where model predicted wrong
for i in range(len(y_test_pred_rfc)):
  if y_test[i] != y_test_pred_rfc[i]:
    tweet = corpus_test[i]
    print(f'y_true {y_test[i]}, y_pred {y_test_pred_rfc[i]}')
    print(tweet)
    print()

In [ ]:
pickle.dump(y_test_proba_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_test_proba_rfc", 'wb'))
pickle.dump(y_test_pred_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_test_pred_rfc", 'wb'))
pickle.dump(y_chopsticks_pred_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_chopsticks_pred_rfc", 'wb'))
pickle.dump(y_all_pred_rfc, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_all_pred_rfc", 'wb'))

loading probability: (3)

In [ ]:
y_test_proba_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_test_proba_rfc', allow_pickle=True)
y_test_pred_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_test_pred_rfc', allow_pickle=True)
y_chopsticks_pred_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_chopsticks_pred_rfc', allow_pickle=True)
y_all_pred_rfc = np.load('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/y_all_pred_rfc', allow_pickle=True)

manual adding predictions: (4)

In [ ]:
df_all

In [ ]:
df_chopsticks

In [ ]:
y_chopsticks_proba_rfc

In [ ]:
df_chopsticks['pred_label'] = y_chopsticks_pred_rfc
df_all['pred_label'] = y_all_pred_rfc

In [ ]:
df_chopsticks['pred_score'] = y_chopsticks_proba_rfc [:,1]
df_all[ 'pred_score'] = y_all_proba_rfc [:,1]

In [ ]:
# Changing object type column to datetime
df_chopsticks['interval_start_date'] = pd.to_datetime(df_chopsticks['interval_start_date'])
df_chopsticks['created_at'] = pd.to_datetime(df_chopsticks['created_at'])
df_chopsticks['date'] = df_chopsticks['created_at'].dt.date # Creating new column with just the date
df_chopsticks['date'] = pd.to_datetime(df_chopsticks['date'])

# get week start date to aggregate over weeks
df_chopsticks['week_start_date'] = df_chopsticks['date'].apply(lambda x: x - datetime.timedelta(days=x.weekday()))
df_chopsticks['week_start_date'] = df_chopsticks['week_start_date'].dt.date
df_chopsticks.head()

In [ ]:
# Changing object type column to datetime
df_all['interval_start_date'] = pd.to_datetime(df_all['interval_start_date'])
df_all['created_at'] = pd.to_datetime(df_all['created_at'])
df_all['date'] = df_all['created_at'].dt.date # Creating new column with just the date
df_all['date'] = pd.to_datetime(df_all['date'])

# get week start date to aggregate over weeks
df_all['week_start_date'] = df_all['date'].apply(lambda x: x - datetime.timedelta(days=x.weekday()))
df_all['week_start_date'] = df_all['week_start_date'].dt.date
df_all.head()

In [ ]:
# positive is going to be defined as greater than 80% prediction score that it is positive (adjustable based on threshold)
df_all_pos = df_all[(df_all['pred_label'] == 4)&(df_all['pred_score']>0.8)]

# negative is going to be defined as greater than 80% prediction score that it is negative (adjustable based on threshold)
df_all_neg = df_all[(df_all['pred_label'] == 0)&(df_all['pred_score']<0.25)]

In [ ]:
# positive is going to be defined as greater than 80% prediction score that it is positive (adjustable based on threshold)
df_chopsticks_pos = df_chopsticks[(df_chopsticks['pred_label'] == 4)&(df_chopsticks['pred_score']>0.9)]

# negative is going to be defined as greater than 80% prediction score that it is negative (adjustable based on threshold)
df_chopsticks_neg = df_chopsticks[(df_chopsticks['pred_label'] == 0)&(df_chopsticks['pred_score']<0.25)]

In [ ]:
df_chopsticks_pos

In [ ]:
pickle.dump(df_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_chopsticks.pkl", "wb")) 
pickle.dump(df_chopsticks_pos, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_chopsticks_pos.pkl", "wb")) 
pickle.dump(df_chopsticks_neg, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_chopsticks_neg.pkl", "wb")) 

pickle.dump(df_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_all.pkl", "wb")) 
pickle.dump(df_all_pos, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_all_pos.pkl", "wb")) 
pickle.dump(df_all_neg, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_all_neg.pkl", "wb")) 


loading predictions: (4)

In [ ]:
df_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_chopsticks.pkl')
df_chopsticks_pos = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_chopsticks_pos.pkl')
df_chopsticks_neg = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_chopsticks_neg.pkl')

df_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_all.pkl')
df_all_pos = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_all_pos.pkl')
df_all_neg = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/df_all_neg.pkl')

In [ ]:
df_chopsticks_pos

manually sorting by week (5):

In [ ]:
# aggregate data and get aggregated statistics
pos_byweek_all = df_all_pos.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for positive tweets, get total retweets each week (sum of num retweets column over each week)
pos_byweek_all.columns = ['date', 'total_retweets']

neg_byweek_all = df_all_neg.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for negative tweets, get total retweets each week (sum of num retweets column over each week)
neg_byweek_all.columns = ['date', 'total_retweets']

In [ ]:
# aggregate data and get aggregated statistics
pos_byweek_chopsticks = df_chopsticks_pos.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for positive tweets, get total retweets each week (sum of num retweets column over each week)
pos_byweek_chopsticks.columns = ['date', 'total_retweets']

neg_byweek_chopsticks = df_chopsticks_neg.groupby(by=['week_start_date']).agg({'retweet_count': ['sum']}).reset_index() # for negative tweets, get total retweets each week (sum of num retweets column over each week)
neg_byweek_chopsticks.columns = ['date', 'total_retweets']

In [ ]:
# for each week, get list of tweets with the highest number of retweets (which you can manually review to see what was trending that week)
top_tweets_peak_weeks_all = df_all.sort_values(by='retweet_count', ascending=False)
top_tweets_peak_weeks_all = top_tweets_peak_weeks_all.groupby('week_start_date').head(10).sort_values(by=['week_start_date', 'retweet_count'], ascending=False)
top_tweets_peak_weeks_all
#top_tweets_peak_weeks.to_csv(os.path.join(data_dir, 'top_tweets_peak_weeks.csv'), index=False)

In [ ]:
# for each week, get list of tweets with the highest number of retweets (which you can manually review to see what was trending that week)
top_tweets_peak_weeks_chopsticks = df_chopsticks.sort_values(by='retweet_count', ascending=False)
top_tweets_peak_weeks_chopsticks = top_tweets_peak_weeks_chopsticks.groupby('week_start_date').head(10).sort_values(by=['week_start_date', 'retweet_count'], ascending=False)
top_tweets_peak_weeks_chopsticks
#top_tweets_peak_weeks.to_csv(os.path.join(data_dir, 'top_tweets_peak_weeks.csv'), index=False)

In [ ]:
pickle.dump(pos_byweek_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_chopsticks.pkl", "wb")) 
pickle.dump(neg_byweek_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_chopsticks.pkl", "wb")) 
pickle.dump(top_tweets_peak_weeks_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/top_tweets_peak_weeks_chopsticks.pkl", "wb")) 

pickle.dump(pos_byweek_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_all.pkl", "wb")) 
pickle.dump(neg_byweek_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_all.pkl", "wb")) 
pickle.dump(top_tweets_peak_weeks_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/top_tweets_peak_weeks_all.pkl", "wb")) 


loading by week: (5)

In [ ]:
pos_byweek_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_chopsticks.pkl')
neg_byweek_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_chopsticks.pkl')
top_tweets_peak_weeks_chopsticks = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/top_tweets_peak_weeks_chopsticks.pkl')


pos_byweek_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_all.pkl')
neg_byweek_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_all.pkl')
top_tweets_peak_weeks_all = pd.read_pickle('C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/top_tweets_peak_weeks_all.pkl')

manual tweet #: (5b)

In [ ]:
countlist = []
for i in range (len(df_all_pos)):
    countlist.append(1)
    
df_all_pos['counting'] = countlist
df_all_pos

pos_byweek_tweets_all = df_all_pos.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
pos_byweek_tweets_all.columns = ['date', '# of tweets']
pos_byweek_tweets_all


In [ ]:
countlist = []
for i in range (len(df_chopsticks_pos)):
    countlist.append(1)
    
df_chopsticks_pos['counting'] = countlist
df_chopsticks_pos

pos_byweek_tweets_chopsticks = df_chopsticks_pos.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
pos_byweek_tweets_chopsticks.columns = ['date', '# of tweets']
pos_byweek_tweets_chopsticks

In [ ]:
countlist = []
for i in range (len(df_chopsticks_neg)):
    countlist.append(1)
    
df_chopsticks_neg['counting'] = countlist
df_chopsticks_neg

neg_byweek_tweets_chopsticks = df_chopsticks_neg.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
neg_byweek_tweets_chopsticks.columns = ['date', '# of tweets']
neg_byweek_tweets_chopsticks

In [ ]:
countlist = []
for i in range (len(df_all_neg)):
    countlist.append(1)
    
df_all_neg['counting'] = countlist
df_all_neg

neg_byweek_tweets_all = df_all_neg.groupby(by=['week_start_date']).agg({'counting': ['sum']}).reset_index() 
neg_byweek_tweets_all.columns = ['date', '# of tweets']
neg_byweek_tweets_all

In [ ]:
pickle.dump(neg_byweek_tweets_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_tweets_chopsticks.pkl", "wb")) 
pickle.dump(neg_byweek_tweets_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_tweets_all.pkl", "wb")) 
pickle.dump(pos_byweek_tweets_chopsticks, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_tweets_chopsticks.pkl", "wb")) 
pickle.dump(pos_byweek_tweets_all, open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_tweets_all.pkl", "wb")) 

loading tweet #: (5b)

In [ ]:
neg_byweek_tweets_chopsticks = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_tweets_chopsticks.pkl", 'rb'))
neg_byweek_tweets_all = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/neg_byweek_tweets_all.pkl", 'rb'))
pos_byweek_tweets_chopsticks = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_tweets_chopsticks.pkl", 'rb'))
pos_byweek_tweets_all = pickle.load(open("C:/Users/flore/Desktop/UTS S5/STEM fellowship/rfc/pos_byweek_tweets_all.pkl", 'rb'))

manual graph making: (6)

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
ax.plot(neg_byweek_all["date"], neg_byweek_all['total_retweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 1000]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
ax.plot(pos_byweek_all["date"], pos_byweek_all['total_retweets'], c='blue', label='Positive tweets')
ax.plot(neg_byweek_all["date"], neg_byweek_all['total_retweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 10000]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()

samething but tweet #: (6b)

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
ax.plot(pos_byweek_tweets_all["date"], pos_byweek_tweets_all['# of tweets'], c='blue', label='Positive tweets')
ax.plot(neg_byweek_tweets_all["date"], neg_byweek_tweets_all['# of tweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 2000]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()

In [ ]:
# plot number of retweets over time (aggregated over each week) for positive and negative tweets
fig, ax = plt.subplots(figsize=(30, 8))
#ax.plot(pos_byweek_tweets_chopsticks["date"], pos_byweek_tweets_chopsticks['# of tweets'], c='blue', label='Positive tweets')
ax.plot(neg_byweek_tweets_all["date"], neg_byweek_tweets_all['# of tweets'], c='red', label='Negative tweets')

"""
# format the vertical lines and annotations
y = {
  0: 20000,
  1: 40000,
  2: 30000,
  3: 60000,
  4: 80000,
  5: 50000,
  6: 40000,
  7: 40000,
  8: 60000,
  9: 80000,
  10: 40000,
  11: 140000,
  12: 120000,
  13: 100000,
  14: 160000,
  15: 160000,
  16: 100000,
  17: 120000,
}
"""

y_offset = 10000
left_align = [1, 4, 5, 8, 9, 10, 15, 16, 17]

"""
# vertical line for major events
for i in range(len(dates)):
  date = dates[i]
  event = events[i]
  if i in left_align:
    align = 'left'
  else:
    align = 'right'
  plt.axvline(x=date, color='grey', linestyle=':')
  ax.text(date, y[i]-y_offset, event, ha=align, size=10)
"""

# Major ticks every 6 months.
fmt_half_year = mdates.MonthLocator(interval=6)
ax.xaxis.set_major_locator(fmt_half_year)

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)

# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Round to nearest half of year.
datemin = np.datetime64(datetime.date(2013, 7, 1), 'm')
datemax = np.datetime64(datetime.date(2021, 7, 1), 'm')
ax.set_xlim(datemin, datemax)

# Format the coords message box, i.e. the numbers displayed as the cursor moves
# across the axes within the interactive GUI.
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.format_ydata = lambda x: f'${x:.2f}'  # Format the price.
ax.grid(True)

# Label the axes
ax.set(xlabel='Date', ylabel='Weekly total retweet count')
ax.set_ylim([0, 250]) # scale the y-axis range

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()
plt.legend() # add the legend

plt.show()